In [1]:
import sys
import unittest
from enum import Enum

In [2]:
class Dijkstras: 
  
    def __init__(self, graph_dictionary):
        self.shortest_distances = {} #shortest_distances = store the shortest distances between any two nodes
        self.shortest_paths = {} #shortest_paths = store the shortest routes between any two nodes (name of the nodes)
        
        for start_node in graph_dictionary.keys(): #calling shortest path algorithm making every node as the start node 
            self.shortest_distances[start_node], self.shortest_paths[start_node] = self.calculate_shortest_routes(graph_dictionary, start_node)

    def calculate_shortest_routes(self, graph_dictionary, start_node): #computing all shortest routes
        shortest_routes = {}
        visit_nodes = {start_node: 0}
        previous_nodes = {}
        
        while visit_nodes:
            next_node_key = min(visit_nodes, key=visit_nodes.get)
            distance_to_next_node = visit_nodes.pop(next_node_key)

            for node_key, node_value in graph_dictionary[next_node_key].items():
                if node_key not in shortest_routes:
                    addup_edges = distance_to_next_node + node_value
                    if node_key not in visit_nodes:
                        visit_nodes[node_key] = addup_edges
                        previous_nodes[node_key] = next_node_key
                    else:
                        if (visit_nodes[node_key] > addup_edges):
                            visit_nodes[node_key] = addup_edges
                            previous_nodes[node_key] = next_node_key

            if next_node_key != start_node or distance_to_next_node > 0: #cannot assign start node with 0 distance 
                shortest_routes[next_node_key] = distance_to_next_node
        return shortest_routes, previous_nodes

    def get_distance(self, start_node, end_node): #shortest distance from a start node to end node 
        if end_node in self.shortest_distances.get(start_node, {}):
            routes = end_node #display
            node_key = end_node
            while (self.shortest_paths[start_node][node_key] != start_node): 
                routes = routes + self.shortest_paths[start_node][node_key]
                node_key = self.shortest_paths[start_node][node_key]
            routes = routes + start_node
            routes = routes[::-1]
            print("\nShortest route from " + start_node + " to " + end_node + " is: " + routes + " with distance " + str(self.shortest_distances[start_node][end_node]))
            
            return self.shortest_distances[start_node][end_node]

        print("\nNO SUCH ROUTE from " + start_node + " to " + end_node)
        return "NO SUCH ROUTE"

In [3]:
class Railroad: #receives the city-to-city railroad & their distances such as "AB5, BC4, CD8, DC8, DE6, AD5, CE2, EB3, AE7" 
    
    TripType = Enum('TripType', 'exact_stops max_stops')

    def __init__(self, graph):
        self.graph = self.convert_graph_to_dictionary(graph)
        self.dijkstras = Dijkstras(self.graph)

    def convert_graph_to_dictionary(self, graph): #converts graph string to dictionary
        graph_dictionary = {}
        connection_list = [x.strip() for x in graph.split(',')]
        for connection in connection_list:
            departure_city = connection[0]
            next_city = connection[1]
            distance = float(connection[2:])

            if departure_city not in graph_dictionary:
                graph_dictionary[departure_city] = {}

            graph_dictionary[departure_city][next_city] = distance
        return graph_dictionary

    def get_shortest_distance(self, departure_city, destination_city): #gets the shortest distance 
        return self.dijkstras.get_distance(departure_city, destination_city)

    def get_distance(self, route): #calculates distance for specific routes 
        if route == "":
            print("\n" + route + ": NO SUCH ROUTE")
            return "NO SUCH ROUTE"

        route_cities = route.split('-')
        current_city = route_cities[0]
        total_distance = 0
        for i in range(1, len(route_cities)): # looping through cities on the route to sum up the distances
            next_city = route_cities[i]
            if current_city in self.graph and next_city in self.graph[current_city]:
                total_distance += self.graph[current_city][next_city]
            else:
                print("\n" + route + ": NO SUCH ROUTE")
                return "NO SUCH ROUTE"
            current_city = next_city

        print("\nThe route " + route + " has the distance of " + str(total_distance))
        return total_distance

    def get_number_of_possible_trips(self, departure_city, destination_city, num_stops, trip_type, callback=1): #possible number of trips from departure to destination city
        trip_number = 0
        if num_stops == 0:
            if (callback == 1):
                if (trip_type == self.TripType.max_stops):
                    print("\nNumber of possible trips from " + departure_city + " to " + destination_city + " with maximum of " + str(num_stops) + " stops are: " + str(trip_number))
                else:
                    print("\nNumber of possible trips from " + departure_city + " to " + destination_city + " with exact " + str(num_stops) + " stops are: " + str(trip_number))
            return 0

        for next_city in self.graph[departure_city]: #all connections from departure city traversal
            if next_city == destination_city and (trip_type == self.TripType.max_stops or num_stops == 1):
                trip_number += 1

            trip_number += self.get_number_of_possible_trips(next_city, destination_city, num_stops - 1, trip_type, 0)
        
        if (callback == 1):
            if (trip_type == self.TripType.max_stops):
                print("\nNumber of possible trips from " + departure_city + " to " + destination_city + " with maximum of " + str(num_stops) + " stops are: " + str(trip_number))
            else:
                print("\nNumber of possible trips from " + departure_city + " to " + destination_city + " with exact " + str(num_stops) + " stops are: " + str(trip_number))

        return trip_number

    def get_number_of_possible_trips_to_maximum_distance(self, departure_city, destination_city, max_distance, callback=1): #possible number of trips from departure to destination city less than maximum distance 
        trip_number = 0
        # iterate through all connections from departure_city
        for next_city, distance_to_next in self.graph[departure_city].items():
            if next_city == destination_city and distance_to_next < max_distance:
                trip_number += 1
            if distance_to_next < max_distance:
                trip_number += self.get_number_of_possible_trips_to_maximum_distance(next_city, destination_city, max_distance - distance_to_next, 0)

        if (callback == 1):
            print("\nNumber of possible trips from " + departure_city + " to " + destination_city + " with maximum distance of " + str(max_distance) + " are: " + str(trip_number))

        return trip_number

In [4]:
Railnetwork = Railroad("AB5, BC4, CD8, DC8, DE6, AD5, CE2, EB3, AE7")

In [5]:
class DijkstrasModuleTest(unittest.TestCase):
    def setUp(self):
        self.railroad = Railnetwork

    # Testing cases of railroad.get_shortest_distance
    def test_route_to_self_should_not_automatically_return_zero_distance(self):
        self.assertEqual(self.railroad.get_shortest_distance("D", "D"), 16)

    def test_shortest_dist_to_unreachable_destination_should_return_no_such_route(self):
        self.assertEqual(self.railroad.get_shortest_distance("C", "A"), "NO SUCH ROUTE")

    def test_direct_connection_shortest_should_return_direct_distance(self):
        self.assertEqual(self.railroad.get_shortest_distance("A", "B"), 5)

    def test_indirect_connection_shortest_should_return_indirect_distance(self):
        self.assertEqual(self.railroad.get_shortest_distance("A", "E"), 7)

    def test_two_routes_tie_should_return_correct_distance(self):
        self.assertEqual(self.railroad.get_shortest_distance("A", "C"), 9)

def suite(): #collecting all shortest path module tests from this module 
    test_suite = unittest.TestSuite()
    test_suite.addTest(unittest.makeSuite(DijkstrasModuleTest))
    return test_suite

if __name__ == '__main__':
    mySuit=suite()

    runner=unittest.TextTestRunner()
    runner.run(mySuit)

.....


Shortest route from A to B is: AB with distance 5.0

Shortest route from A to E is: AE with distance 7.0

Shortest route from D to D is: DCD with distance 16.0

NO SUCH ROUTE from C to A

Shortest route from A to C is: ABC with distance 9.0



----------------------------------------------------------------------
Ran 5 tests in 0.015s

OK


In [6]:
class GetDistanceModuleTest(unittest.TestCase):
    def setUp(self):
        self.railroad = Railnetwork

    # Testing cases of railroad.get_distance
    def test_empty_route_should_return_no_such_route(self):
        self.assertEqual(self.railroad.get_distance(""), "NO SUCH ROUTE")

    def test_nonexistant_single_step_route_should_return_no_such_route(self):
        self.assertEqual(self.railroad.get_distance("A-C"), "NO SUCH ROUTE")

    def test_existing_single_step_route_should_return_distance(self):
        self.assertAlmostEqual(self.railroad.get_distance("A-B"), 5)

    def test_nonexistant_multiple_step_route_should_return_no_such_route(self):
        self.assertEqual(self.railroad.get_distance("A-B-D"), "NO SUCH ROUTE")

    def test_existing_multiple_step_route_should_return_distance(self):
        self.assertAlmostEqual(self.railroad.get_distance("A-B-C"), 9)

def suite():
    test_suite = unittest.TestSuite()
    test_suite.addTest(unittest.makeSuite(GetDistanceModuleTest))
    return test_suite
	
if __name__ == '__main__':
    mySuit=suite()

    runner=unittest.TextTestRunner()
    runner.run(mySuit)

.....


: NO SUCH ROUTE

The route A-B-C has the distance of 9.0

The route A-B has the distance of 5.0

A-B-D: NO SUCH ROUTE

A-C: NO SUCH ROUTE



----------------------------------------------------------------------
Ran 5 tests in 0.009s

OK


In [7]:
class GetTripsNumberModuleTest(unittest.TestCase):
    def setUp(self):
        self.railroad = Railnetwork

    # Testing cases of railroad.get_number_of_possible_trips
    def test_number_trips_max_stops_with_missing_connection_should_return_zero(self):
        self.assertEqual(self.railroad.get_number_of_possible_trips("A", "A", 3, Railroad.TripType.max_stops), 0)

    def test_number_trips_max_stops_with_valid_routes_should_return_number(self):
        self.assertEqual(self.railroad.get_number_of_possible_trips("E", "D", 5, Railroad.TripType.max_stops), 2)

    def test_number_trips_max_stops_with_too_small_stops_should_return_zero(self):
        self.assertEqual(self.railroad.get_number_of_possible_trips("E", "D", 2, Railroad.TripType.max_stops), 0)

    def test_number_trips_exact_stops_with_missing_connection_should_return_zero(self):
        self.assertEqual(self.railroad.get_number_of_possible_trips("A", "A", 3, Railroad.TripType.exact_stops), 0)

    def test_number_trips_exact_stops_with_valid_routes_should_return_number(self):
        self.assertEqual(self.railroad.get_number_of_possible_trips("A", "C", 2, Railroad.TripType.exact_stops), 2)

    def test_number_trips_exact_stops_with_no_matches_should_return_zero(self):
        self.assertEqual(self.railroad.get_number_of_possible_trips("E", "D", 4, Railroad.TripType.exact_stops), 0)

    def test_num_routes_of_less_than_zero_distance_should_return_zero(self): #test cases of number of maxumum possible trips to maximum distance 
        self.assertEqual(self.railroad.get_number_of_possible_trips_to_maximum_distance("E", "B", 0), 0)

    def test_num_routes_to_unreachable_city_should_return_zero(self):
        self.assertEqual(self.railroad.get_number_of_possible_trips_to_maximum_distance("E", "A", 100), 0)

    def test_num_routes_of_complex_path_should_return_five(self):
        self.assertEqual(self.railroad.get_number_of_possible_trips_to_maximum_distance("A", "C", 19), 5)

def suite():
    test_suite = unittest.TestSuite()
    test_suite.addTest(unittest.makeSuite(GetTripsNumberModuleTest))
    return test_suite

if __name__ == '__main__':
    mySuit=suite()

    runner=unittest.TextTestRunner()
    runner.run(mySuit)

.........


Number of possible trips from A to C with maximum distance of 19 are: 5

Number of possible trips from E to B with maximum distance of 0 are: 0

Number of possible trips from E to A with maximum distance of 100 are: 0

Number of possible trips from A to A with exact 3 stops are: 0

Number of possible trips from E to D with exact 4 stops are: 0

Number of possible trips from A to C with exact 2 stops are: 2

Number of possible trips from A to A with maximum of 3 stops are: 0

Number of possible trips from E to D with maximum of 2 stops are: 0

Number of possible trips from E to D with maximum of 5 stops are: 2



----------------------------------------------------------------------
Ran 9 tests in 0.022s

OK


In [8]:
class ReceptivitiTest(unittest.TestCase):
    def setUp(self):
        self.railroad = Railnetwork

    def test_01(self):
        self.assertAlmostEqual(self.railroad.get_distance("A-B-C"), 9)

    def test_02(self):
        self.assertAlmostEqual(self.railroad.get_distance("A-D"), 5)

    def test_03(self):
        self.assertAlmostEqual(self.railroad.get_distance("A-D-C"), 13)

    def test_04(self):
        self.assertAlmostEqual(self.railroad.get_distance("A-E-B-C-D"), 22)

    def test_05(self):
        self.assertEqual(self.railroad.get_distance("A-E-D"), "NO SUCH ROUTE")

    def test_06(self):
        self.assertEqual(self.railroad.get_number_of_possible_trips("C", "C", 3, Railroad.TripType.max_stops), 2)

    def test_07(self):
        self.assertEqual(self.railroad.get_number_of_possible_trips("A", "C", 4, Railroad.TripType.exact_stops), 3)

    def test_08(self):
        self.assertEqual(self.railroad.get_shortest_distance("A", "C"), 9)

    def test_09(self):
        self.assertEqual(self.railroad.get_shortest_distance("B", "B"), 9)

    def test_10(self):
        self.assertEqual(self.railroad.get_number_of_possible_trips_to_maximum_distance("C", "C", 30), 7)

def suite():
    test_suite = unittest.TestSuite()
    test_suite.addTest(unittest.makeSuite(ReceptivitiTest))
    return test_suite

if __name__ == '__main__':
    mySuit=suite()

    runner=unittest.TextTestRunner()
    runner.run(mySuit)

..........


The route A-B-C has the distance of 9.0

The route A-D has the distance of 5.0

The route A-D-C has the distance of 13.0

The route A-E-B-C-D has the distance of 22.0

A-E-D: NO SUCH ROUTE

Number of possible trips from C to C with maximum of 3 stops are: 2

Number of possible trips from A to C with exact 4 stops are: 3

Shortest route from A to C is: ABC with distance 9.0

Shortest route from B to B is: BCEB with distance 9.0

Number of possible trips from C to C with maximum distance of 30 are: 7



----------------------------------------------------------------------
Ran 10 tests in 0.058s

OK
